<a href="https://colab.research.google.com/github/nhonlvsoict/MachineLearning-DataMiningProject/blob/main/%5BTraining_Triplet%5D_Facedetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y keras-nightly
!pip install tensorflow==1.15.0
!pip install keras==2.1.5
!pip install h5py==2.10.0
!pip install tensorflow_gpu==1.15.0

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900
     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 3.8MB 38.7MB/s 
     |████████████████████████████████| 512kB 32.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=bb11db2ab261f423a10f24c0c63ffc4198221cd196847feaf7cbe6ee8fd62c4a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorboard 2.5.0
   

In [3]:
import numpy as np
import cv2

In [4]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/X_train_triplet1.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
    X=X[2400:]
#     X = np.expand_dims(X, axis=3)
    X = X/255.
with open('/content/drive/MyDrive/Colab Notebooks/y_train_triplet1.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)
    y=y[2400:]

In [ ]:



import gc

gc.collect()


In [ ]:
np.shape(X), np.shape(y)


In [7]:
# import sklearn.preprocessing

# label_binarizer = sklearn.preprocessing.LabelBinarizer()

# label_binarizer.fit(range(4))

# y = label_binarizer.transform(y)
# y_val = label_binarizer.transform(y_val)

# print(y.shape)

In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import applications

In [9]:

import tensorflow as tf
from tensorflow.keras import backend as K
import h5py
import tensorflow.keras.applications.vgg16 as vgg16

def convnet_model_():
    vgg_model = vgg16.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
#     x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model

In [10]:
deep_rank_model = deep_rank_model()
deep_rank_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 221, 221, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
___________

In [11]:
batch_size = 24

_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)

In [12]:
deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))

In [13]:
def image_batch_generator(images, labels, batch_size):
    labels = np.array(labels)
    while True:
        batch_paths = np.random.choice(a = len(images), size = batch_size//3)
        input_1 = []
        
        for i in batch_paths:
            pos = np.where(labels == labels[i])[0]
            neg = np.where(labels != labels[i])[0]
            
            j = np.random.choice(pos)
            while j == i:
                j = np.random.choice(pos)
             
            k = np.random.choice(neg)
            while k == i:
                k = np.random.choice(neg)
            
            input_1.append(images[i])
            input_1.append(images[j])
            input_1.append(images[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/drive/MyDrive/Colab Notebooks/triplet_weight1.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
from tensorflow.keras.models import load_model

print("Loading pre-trained weight")
deep_rank_model.load_weights('/content/drive/MyDrive/Colab Notebooks/triplet_weight1.hdf5')

Loading pre-trained weight


In [ ]:

deep_rank_model.fit_generator(generator=image_batch_generator(X, y, batch_size),
                   steps_per_epoch=len(X)//batch_size,
                   epochs=2000,
                   verbose=1,
                   initial_epoch= 528,
                   callbacks=callbacks_list)
     

Epoch 529/2000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
248/249 [============================>.] - ETA: 0s - loss: 0.2983
Epoch 00529: loss improved from inf to 0.29765, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight1.hdf5
249/249 [==============================] - 111s 447ms/step - loss: 0.2977
Epoch 530/2000
248/249 [============================>.] - ETA: 0s - loss: 0.2822
Epoch 00530: loss improved from 0.29765 to 0.28346, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight1.hdf5
249/249 [==============================] - 102s 411ms/step - loss: 0.2835
Epoch 531/2000
248/249 [============================>.] - ETA: 0s - loss: 0.2759
Epoch 00531: loss improved from 0.28346 to 0.27541, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight1.hdf5
249/249 [==============================] - 104s 419ms/step - loss: 0.2754
Epoch 532/2000
248/249 [============================>.] - ETA

In [ ]:
from tqdm import tqdm
    embs128 = []
    for x in tqdm(X):
        image = x/255
        image = np.expand_dims(image, axis=0)
        emb128 = deep_rank_model.predict([image, image, image])
        embs128.append(emb128[0])
        del image"

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/name_map1.pkl', 'rb') as f:
    names = pickle.load(f)
    names = np.array(y)
    names=names[2400:]
np.shape(names)    

In [ ]:
import time
import dlib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:

image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/lfw/Zico/Zico_0002.jpg')


# Khai báo việc sử dụng các hàm của dlib
hog_face_detector = dlib.get_frontal_face_detector()

faces_hog = hog_face_detector(image, 1)

for face in faces_hog:
    x = face.left()
    y = face.top()
    w = face.right() - x
    h = face.bottom() - y
    frame = image[y:y+h, x:x+w]
    frame = cv2.resize(frame, (221, 221))
    frame = frame /255.
    frame = np.expand_dims(frame, axis=0)
    emb128 = deep_rank_model.predict([frame, frame, frame])
    
    minimum = 99999
    person = -1
    for k, e in enumerate(embs128):
        #Euler distance
        dist = np.linalg.norm(emb128-e)
        if dist < minimum:
            minimum = dist
            person = k
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.putText(image, str(name[person]), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
cv2_imshow(image)

In [ ]:

image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/lfw/Yoko_Ono/Yoko_Ono_0002.jpg')


# Khai báo việc sử dụng các hàm của dlib
hog_face_detector = dlib.get_frontal_face_detector()

faces_hog = hog_face_detector(image, 1)

for face in faces_hog:
    x = face.left()
    y = face.top()
    w = face.right() - x
    h = face.bottom() - y
    frame = image[y:y+h, x:x+w]
    frame = cv2.resize(frame, (221, 221))
    frame = frame /255.
    frame = np.expand_dims(frame, axis=0)
    emb128 = deep_rank_model.predict([frame, frame, frame])
    minimum = 99999
    person = -1
    for k, e in enumerate(embs128):
        #Euler distance
        dist = np.linalg.norm(emb128-e)
        if dist < minimum:
            minimum = dist
            person = k
            # print(dist)
    print(person)
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.putText(image, str(names[person]), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
cv2_imshow(image)

In [ ]:
np.shape(names)